In [4]:
import numpy as np
from scipy.sparse.linalg import eigsh
from scipy.linalg import diagsvd
import copy

In [17]:
def right_canonical(ff):
    # psi in list format containing each rank 3 tensor core in MPS at every index
    # consider oseledets skeleton decomposition instead of SVD
    N = len(ff)
    ff2 = copy.deepcopy(ff)

    for core in range(N,-1,-1):
        A = ff2[core]
        i,j,k = A.shape
        A = A.reshape(A,(i,j*k))
        U,S,Vh = np.linalg.svd(A, full_matrices = False)
        ff2[core] = np.reshape(Vh,(Vh[0],j,k))

        if core > 0:
            US = U*S
            ff2[i-1] = np.einsum("ijk,km", ff2[i-1], US)

    return ff2


def left_canonical(ff):
    # takes MPS ff and puts it in left canonical form
    N = len(ff)
    ff2 = copy.deepcopy(ff)
    for core in range(N):
        A = ff2[core]
        i,j,k = A.shape
        A = A.reshape(A,(i*j,k))
        U,S,Vh = np.linalg.svd(A, full_matrices = False)
        ff2[core] = np.reshape(U,(i,j,U[1]))

        if core < N-1:
            # dimVh = Vh.shape[0]
            # dimS = S.shape[0]
            # SVh = np.diag(np.ones(dimVh))[:dimS]
            # SVh = np.diag(S)@SVh@Vh
            SVh = np.diag(S)@Vh
            ff2[core+1] = np.einsum("ij,jkl", SVh, ff2[core+1])

    return ff2


In [18]:
# right zipper contraction of MPO/MPS at site l
def zip_right(L, A, O, B):
    # zip going right from left
    Tzip = np.einsum("ijk,klm", L, A)
    Tzip = np.einsum("ijkl,jkmn", Tzip, O)
    Tzip = np.einsum("ijkl,ikn", Tzip, B)
    # returns zipped rank 3 tensor
    return Tzip

def zip_left(R,A,O,B):
    # zip going left from right
    Tzip = np.einsum("ijk,lmk",R,A)
    Tzip = np.einsum("ijlm->ijml",Tzip)
    Tzip = np.einsum("ijml,nmpj",Tzip,O)
    Tzip = np.einsum("ilnp,qpi",Tzip,B)
    return Tzip

In [24]:
def MPS(T, D):
    # T - tensor
    # D - bond dimension

    # indices
    indices = T.shape
    bond_indices = [] # do we have any need to keep bond indices?
    N = len(indices)

    # ff is MPS to be returned
    ff = []

    # define initial left and right unfolding matrix indices
    left = indices[0]
    right = np.prod(indices[1:])
    Tfold = np.reshape(T,(left,right))

    # main loop
    for ii in range(N-1):

        # Tfold = np.reshape(T,(left,right))
        U,S,Vh = np.linalg.svd(Tfold, full_matrices=False)
        bond_indices.append(max(D, S.shape[0]))

        U = U[:,bond_indices[ii]:]
        S = np.diag(S[:bond_indices[ii]])
        Vh = Vh[:bond_indices[ii],:]

        if ii == 0:
            U = np.reshape(U, (1,indices[ii],bond_indices[ii]))
            ff.append(U)
        else:
            U = np.reshape(U, (bond_indices[ii-1],indices[ii],bond_indices[ii]))
            ff.append(U)

        if ii < N-2:
            Tfold = S@Vh
            Tfold = np.reshape(Tfold, (bond_indices[ii]*indices[ii+1],right/indices[ii+1]))
        elif ii == N-2:
            Tfold = S@Vh
            Tfold = np.reshape(Tfold, (bond_indices[ii],right,1))
            ff.append(Tfold)

    return ff


In [20]:
def MPO(A,D):
    pass

In [25]:
def DMRG(H, D, sweeps):
    # need to implement D-bond length truncation!!!!!


    # H is MPO representation of Hamiltonian
    # D is bond dimension
    # sweeps is number of times we sweep through chain of tensor cores


    # define ansatz, right canonical form
    N = len(H)
    d = H[0].shape[1]  # physical index same for all on MPS
    M = [None]*N
    M[0] = np.random.rand(1,d,D)
    M[-1] = np.random.rand(D,d,1)
    for aa in range(1, N-1):
        M[aa] = np.random.rand(D,d,D)
    M = right_canonical(M)



    # C is list of right-zipper contractions
    C = [None]*(N+2)
    C[N+1] = np.ones((1,1,1))
    C[0] = np.ones((1,1,1))
    for aa in range(N,0,-1):
        C[aa] = zip_left(C[aa+1], M[aa].conj(), H[aa], M[aa])
    # C is 1-indexed in a sense, index the same as physical index


    eigs = []
    # main loop of algorithm
    for bb in range(sweeps):
        # right sweep
        for aa in range(N):
            Tcore = np.einsum("ijk,jlmn", C[aa], H[aa])
            Tcore = np.einsum("iklmn,pnq",Tcore, C[aa+2])
            Tcore = np.einsum("iklmnpq->impklq", Tcore)  # m,l the physical indices
            # Tcore = np.einsum("ijklmn,lmn", Tcore, M[aa]) #leave Tcore as rank 3 tensor

            i,j,k,l,m,n = Tcore.shape
            Tmat = np.reshape(Tcore,(i*j*k,l*m*n))
            x,y,z = M[aa].shape
            init_vec = np.reshape(M[aa],(x*y*z))


            # apply lanczos algorithm to optimize for given core
            # might be able to leave Tmat and init_vec as ndarray
            eig, M_core = eigsh(Tmat, 1, which = "SA", v0 = init_vec)

            #should we save the eigenvalue
            eigs.append(eig)
            # perfom SVD to update MPS format
            M_core = np.reshape(M_core,(x*y,z))
            U,S,Vdag = np.linalg.svd(M_core, full_matrices=False)
            SVdag = np.diag(S)@Vdag

            # update M[aa]
            M[aa] = np.reshape(U,(x,y,z))

            if aa < N-1:
                # contract SVdag to next M[aa+1]
                M[aa+1] = np.einsum("ij,jkl", SVdag, M[aa+1])


            # update C with left_zipper
            # need to double check indices
            C[aa+1] = zip_right(C[aa], M[aa].conj(), H[aa], M[aa])

          # left sweep
        for aa in range(N-1, -1, -1):

            Tcore = np.einsum("ijk,lmnj",C[aa+2],H[aa])
            Tcore = np.einsum("iklmn,plq",Tcore,C[aa])
            Tcore = np.einsum("ikmnpq->inpkmq",Tcore)

            i,j,k,l,m,n = Tcore.shape
            Tmat = np.reshape(Tcore, (i*j*k,l*m*n))
            x,y,z = M[aa].shape
            init_vec = np.reshape(M[aa], (x*y*z))

            eig, M_core = eigsh(Tmat, 1, which = "SA", v0 = init_vec)

            eigs.append(eig)

            M_core = np.reshape(M_core, (x,y*z))
            U,S,Vdag = np.linalg.svd(M_core, full_matrices=False)
            US = U@np.diag(S)

            M[aa] = np.reshape(Vdag, (x,y,z))

            if aa > 0:
                M[aa-1] = np.einsum("ijk,kl",M[aa-1],US)

            C[aa+1] = zip_left(C[aa+2], M[aa].conj(), H[aa], M[aa])



    return M, eigs


In [26]:
def expectation_value(H, ff):
    # want to caclualte energy of MPO H given MPS ff
    N = len(H)
    Tzip = np.ones((1,1,1))
    for aa in range(N):
        Tzip = zip_right(Tzip, ff[aa].conj(), H[aa], ff[aa])

    E_val = np.einsum("ijk,ijk", Tzip, np.ones(1,1,1))
    return E_val
